In [1]:
import json
import csv
from datetime import datetime
from datetime import timedelta
import requests

In [12]:
def collectUniqueGroupIdentifierWithType():
    csvfile = open('../HDH-Entities.csv', encoding='utf8', errors='ignore');
    csvReader = csv.reader(csvfile, delimiter=';')

    returnObject = {};
    
    for row in csvReader:
        groupIdentifier = row[2];
        if groupIdentifier != "GroupIdentifier":
            returnObject[groupIdentifier] = row[4];
    csvfile.close()
    return returnObject


def fetchDataByGroupIdentifier(groupIdentifier, dateGte):
    endpoint =  "https://staging.dashboard.heidenheim.de/api/historical/?datetime__gte={0}&groupIdentifier={1}".format(dateGte, groupIdentifier);
    #return endpoint
    response = requests.get(endpoint)
    return response.json()


In [16]:
groupIdentifiers = collectUniqueGroupIdentifierWithType();
print("Object:", groupIdentifiers);
print("Keys:", list(groupIdentifiers.keys()));
print("Values:", list(groupIdentifiers.values()));

Object: {'b84fcd90-f870-4f75-8793-e7ba7526b07f': 'BycicleBox', 'c5cf1f80-bfcd-41fd-b874-1bf763a786c1': 'ParkingSensor', '4d56e281-b89b-4be2-9468-2e70ceb6d876': 'TrafficFrequency', 'd18db740-d364-43ff-86c7-9655d009a529': 'TrafficFrequency', '3d04bfd6-5f14-45bf-bc31-98deaef5ea54': 'TrashCan', '80a8b019-d29a-45f9-9546-d4bff59c3bee': 'WaterLevel', '78e89a84-b206-4b75-96a4-3aca3f722aa0': 'WeatherSensor', 'd005a790-eded-4cde-a090-06a55c3874d5': 'WeatherStation'}
Keys: ['b84fcd90-f870-4f75-8793-e7ba7526b07f', 'c5cf1f80-bfcd-41fd-b874-1bf763a786c1', '4d56e281-b89b-4be2-9468-2e70ceb6d876', 'd18db740-d364-43ff-86c7-9655d009a529', '3d04bfd6-5f14-45bf-bc31-98deaef5ea54', '80a8b019-d29a-45f9-9546-d4bff59c3bee', '78e89a84-b206-4b75-96a4-3aca3f722aa0', 'd005a790-eded-4cde-a090-06a55c3874d5']
Values: ['BycicleBox', 'ParkingSensor', 'TrafficFrequency', 'TrafficFrequency', 'TrashCan', 'WaterLevel', 'WeatherSensor', 'WeatherStation']


In [17]:
todayPlus5Days = (datetime.now() + timedelta(days=5) ).strftime('%Y-%m-%d')
todayPlus5DaysInFormat = todayPlus5Days + "T09:16:04.498Z"
print(todayPlus5DaysInFormat)

2024-03-18T09:16:04.498Z


In [18]:
entities = [];

for groupIdentifier, type in groupIdentifiers.items():
    response = fetchDataByGroupIdentifier(groupIdentifier, todayPlus5DaysInFormat)
    for entity in response['data']:
        entities.append({
            'id': entity['id'],
            'name': entity['name'],
            'labelIdentifier': entity['labelIdentifier'],
            'latitude': entity['latitude'],
            'longitude': entity['longitude'],
            'groupIdentifier': entity['groupIdentifier'],
            'type': type
        });

In [19]:
print(entities[0:1]);

[{'id': 'a5713e95-d456-4bc4-b150-710e455f433c', 'name': 'Heidenheim, Bahnhof, Sammelgarage', 'labelIdentifier': 'unit405', 'latitude': 48.67968, 'longitude': 10.156297, 'groupIdentifier': 'b84fcd90-f870-4f75-8793-e7ba7526b07f', 'type': 'BycicleBox'}]


In [22]:
with open('location-data.json', 'w') as fp:
    json.dump(entities, fp, indent=2)

with open('location-data.js', 'w') as fp:
    jsonData = json.dumps(entities, indent = 2)

    fp.write("window.LOCATION_DATA = " + jsonData)